In [12]:
import numpy as np
import cvxpy as cp
import mosek
import time
import scipy.stats
#from sklearn.model_selection import train_test_split

import phi_divergence as phi
import robust_sampling as rs
import itertools as itertools
#import dataio
#import util

In [22]:
indices = np.asarray(list((itertools.product(np.arange(4), repeat = 2))))
indices

array([[0, 0],
       [0, 1],
       [0, 2],
       [0, 3],
       [1, 0],
       [1, 1],
       [1, 2],
       [1, 3],
       [2, 0],
       [2, 1],
       [2, 2],
       [2, 3],
       [3, 0],
       [3, 1],
       [3, 2],
       [3, 3]])

In [97]:

def make_center(lb,up,m):
    delta = (up-lb)/m
    center = np.arange(lb+delta/2,up,delta)
    return(center)
            

def Freq (cpt,data,lb,up):
    Freq = np.zeros(len(cpt))
    delta = (up-lb)/len(cpt)
    for i in range(len(data)):
        index = int(np.floor((data[i]-lb)/delta))
        Freq[index] = Freq[index] + 1
    return(Freq/len(data))


    
def rob_prob(Omega,a,b):
    d = len(a[0])
    x = cp.Variable(d, nonneg = True)
    z = cp.Variable(d)
    w = cp.Variable(d)
    constraints = [cp.norm(z,1)+Omega*cp.norm(w,2) + a[0] @ x <= b]
    for i in range(d):
        constraints.append(z[i] + w[i] == -a[i] @ x)
    obj = cp.Maximize(cp.sum(x))
    prob = cp.Problem(obj,constraints)
    prob.solve(solver = cp.MOSEK)
    return(x.value)

def lower_bound(alpha,p,S):
    N_v = len(p)
    q = cp.Variable(N_v, nonneg = True)
    t = cp.Variable(N_v, nonneg = True)
    r = 1/N_v*scipy.stats.chi2.ppf(1-alpha, N_v-1)
    constraints = [cp.sum(q) == 1]
    f_obj = 0
    for i in range(N_v):
        if S[i] != 0:
            f_obj = f_obj + q[i]
        z = cp.vstack([q[i]-p[i],(t[i]-q[i])/2])
        constraints.append(cp.norm(z,2) <= (t[i]+q[i])/2)
    constraints.append(cp.sum(t) <= r)
    obj = cp.Minimize(f_obj)
    prob = cp.Problem(obj,constraints)
    prob.solve(solver = cp.MOSEK)
    return(prob.value)
    
    
def cpt_feas (cpt_arr,x,a,b,indices):
    d = len(cpt_arr)
    m = len(cpt_arr[0])
    S = np.zeros(len(indices))
    for i in range(len(S)):
        const = a[0]
        for j in range(d):
            const = const + cpt_arr[j][indices[i][j]] * a[j]
            if const.dot(x) <= b:
                S[i] = 1
    return(S)
    
    
def Ishan(d,alpha,beta,N,Omega,step,a,b):
    xi = np.random.uniform(size = (d,N))*2-1
    cpt_arr = []
    lb = -1
    up = 1
    m = 10
    delta = 0
    indices = np.asarray(list((itertools.product(np.arange(m), repeat = d))))
    p = np.zeros(len(indices))
    freq_ct = []
    for i in range(d):
        cpt_arr.append(make_center(lb,up,m))
        freq_ct.append(Freq(cpt_arr[i], xi[i],-1,1))
    for j in range(len(indices)):
        p[j] = 1
        for k in range(d):
            p[j] = p[j] * freq_ct[k][indices[j][k]]
    lowerbound = -np.inf
    while lowerbound < beta:
        x = rob_prob(Omega,a,b)
        S = cpt_feas (cpt_arr,x,a,b,indices)
        lowerbound = lower_bound(alpha,p,S)
        print('Omega', Omega)
        print('lowerbound',lowerbound)
        print('objective', np.sum(x))
        Omega = Omega + step
        
        
    

In [96]:
a = [np.array([1,1]), np.array([1,0]), np.array([0,1])]
b = 10
Omega = 0
step = 0.14
alpha = 0.001
beta = 0.8
d = 2
N = 100
Ishan(d,alpha,beta,N,Omega,step,a,b)

1.0
Omega 0
lowerbound 0.10181782814003962
objective 10.0
Omega 0.14
lowerbound 0.14335114478050168
objective 8.771929746083186
Omega 0.28
lowerbound 0.14335114478050168
objective 7.8124999493638025
Omega 0.42000000000000004
lowerbound 0.20157137257090674
objective 7.042253484170046
Omega 0.56
lowerbound 0.2457959453274056
objective 6.4102563834657715
Omega 0.7000000000000001
lowerbound 0.30135483613572567
objective 5.88235288365994
Omega 0.8400000000000001
lowerbound 0.30135483613572567
objective 5.434782543549054
Omega 0.9800000000000001
lowerbound 0.9999999954503308
objective 5.050505000569173
